In [1]:
from IPython import get_ipython
from IPython.display import display

In [2]:
!pip install transformers newspaper3k nltk jinja2 ipywidgets langchain PyPDF2
!pip install transformers newspaper3k nltk Jinja2 lxml_html_clean

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 37.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=98864ee4b4bb818b907443398f27bd01f774ea4a86fd61bc38916d66686ff734
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import display, HTML, clear_output
from ipywidgets import interact_manual, Text, Output, HBox, VBox, Button, FileUpload

In [4]:
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
from transformers import pipeline
from newspaper import Article
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from jinja2 import Template
import PyPDF2
import io

In [6]:
def extract_text_from_pdf(pdf_file):
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
    return text

In [10]:
def get_financial_news_summary(text_or_url, is_pdf=False):
    if is_pdf:
        text = text_or_url
    else:
        article = Article(text_or_url)
        article.download()
        article.parse()
        article.nlp()
        text = article.text

    max_text_length = 1024
    text = text[:max_text_length]

    summarizer = pipeline("summarization")
    summary = summarizer(text, max_length=150, min_length=30, do_sample=False)

    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)

    max_text_length = 1024
    text = text[:max_text_length]

    summarizer = pipeline("summarization")
    summary = summarizer(text, max_length=150, min_length=30, do_sample=False)

    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    overall_sentiment = "Neutral"
    if sentiment['compound'] >= 0.05:
        overall_sentiment = "Positive"
    elif sentiment['compound'] <= -0.05:
        overall_sentiment = "Negative"

    source = text_or_url if not is_pdf else "Uploaded PDF"

    return {
        "summary": summary[0]['summary_text'],
        "sentiment": sentiment,
        "overall_sentiment": overall_sentiment,
        "source": source,
    }


In [11]:
def display_summary(text_or_url="", is_pdf=False):
    clear_output(wait=True)
    global news_summary
    news_summary = get_financial_news_summary(text_or_url, is_pdf)

    html_template = Template("""
    <div style="border: 2px solid #007bff; padding: 20px; border-radius: 10px; background-color: #f8f9fa; box-shadow: 2px 2px 5px #888888;">
        <h2 style="color: #007bff; text-align: center;">Financial News Summary</h2>
        <p style="font-weight: bold; color: black;">Source: <span style="color: black;">{{ news_summary.source }}</span></p>
        <h3 style="color: #343a40;">Summary:</h3>
        <p style="text-align: justify; color: black;">{{ news_summary.summary }}</p>
        <h3 style="color: #343a40;">Detailed Sentiment Analysis:</h3>
        <ul style="color: black;">
            <li>Positive: {{ '{:.0%}'.format(news_summary.sentiment.pos) }}</li>
            <li>Negative: {{ '{:.0%}'.format(news_summary.sentiment.neg) }}</li>
            <li>Neutral: {{ '{:.0%}'.format(news_summary.sentiment.neu) }}</li>
        </ul>
    </div>
    """)

    html_content = html_template.render(news_summary=news_summary)
    display(HTML(html_content))

In [12]:
url_input = Text(placeholder="Paste article URL here", layout={'width': '50%'})
pdf_upload = FileUpload(accept='.pdf', multiple=False, layout={'width': '50%'})

def on_submit_url(b):
    display_summary(url_input.value)

def on_upload_pdf(change):
    if change['new']:
        uploaded_file = list(change['new'].values())[0]
        pdf_text = extract_text_from_pdf(io.BytesIO(uploaded_file['content']))
        display_summary(pdf_text, is_pdf=True)


url_button = Button(description="Submit URL")
url_button.on_click(on_submit_url)

pdf_upload.observe(on_upload_pdf, names='value')

display(HBox([url_input, url_button]))
display(VBox([pdf_upload]))

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
